In [1]:
%pip install ultralytics
%pip install super-gradients
%pip install supervision
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.3/664.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 47.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 11.0 M

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 383.5 kB/s eta 0:00:00


In [2]:
from ultralytics import NAS

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from super_gradients.training import models
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

best_model = models.get(
    'yolo_nas_l',
    num_classes=1,
    checkpoint_path="/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth"
).to(DEVICE)


The console stream is logged into /root/sg_logs/console.log


[2024-01-04 14:21:56] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-01-04 14:21:56] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-01-04 14:22:02] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-01-04 14:22:02] WARNING - export.py - Failed to import pytorch_quantization
[2024-01-04 14:22:02] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-01-04 14:22:02] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-01-04 14:22:02] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-01-04 14:22:02] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-01-04 14:22:02] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not fo

In [ ]:
import supervision as sv
import ultralytics

def get_inference(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )

  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections
    nome_imagem = image_name.split('/')[7] + ".txt"
    with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/inferencias/{nome_imagem}","w") as file:
      file.write(str(detections.xyxy))

get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO -

In [5]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

images = []
titles = []

detections_list = []
prediction_list = []

def get_ioudata(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )
  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

  keys = list(ds.images.keys())
  for key in keys:
      # Para anotações
      annotation_detections = ds.annotations[key]
      for detection in annotation_detections:
          detections_list.append(detection[0])

      # Para previsões
      prediction_detections = predictions[key]
      prediction_list.append(prediction_detections)

get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-04 14:22:47] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:50] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:50] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-04 14:22:50] INFO -

In [9]:
import numpy as np


def iou_xyxy(detections, predictions):

  def iou(box1,box2):
    # Check if the bounding boxes are valid.
    if box1[2] is None or box2[2] is None:
      return 0.0

    # Calculate the intersection area.
    inter_top = max(box1[1], box2[1])
    inter_bottom = min(box1[3], box2[3])
    inter_left = max(box1[0], box2[0])
    inter_right = min(box1[2], box2[2])
    inter_area = (inter_right - inter_left) * (inter_bottom - inter_top)

    # Calculate the union area.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    # Calculate the IoU.
    iou = inter_area / union_area
    return iou


  for box2 in predictions:
    print(detections)
    print(box2[0])
    return iou(detections,box2[0])


for i in range(len(detections_list)):
  iou_score = iou_xyxy(detections_list[i], prediction_list[i])
  with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'a') as file:
    file.write(str(iou_score)+ '\n')
  print(iou_score)

[     214.44       32.22         305      112.22]
[     214.18      31.396      294.17      102.93]
0.7692672
[     263.85      143.85      403.08      293.85]
[      262.8         142       404.6       295.2]
0.96136683
[      75.17       65.52      128.27         120]
[     73.938      64.838       128.8      122.24]
0.9186375
[     106.55       106.9      172.07      181.04]
[     102.38      105.79      168.61      184.27]
0.8434799
[     174.83      108.96      215.17      149.65]
[     173.25      107.97      215.07      151.11]
0.9056891
[     184.62      318.46       242.3      384.61]
[      183.5       314.8       237.2       385.2]
0.8446695
[     202.31      263.85      275.39      393.08]
[      193.3       259.2       279.4         388]
0.7916934
[     143.79      105.51      182.41      138.96]
[      144.9       107.1      182.88       137.9]
0.8845069
[     327.69       272.3      383.07      323.08]
[      325.2       269.6       382.4       327.2]
0.83443916
None
[  

IndexError: ignored

-0.28760698
[     128.46      215.39      216.92      280.77]
[      167.2       335.2         269       413.2]
-0.16471283
[     55.865       47.59      129.32      148.97]
[     296.16      388.54      414.13      513.83]
-2.252757
[     256.92      271.54      325.38      326.92]
[     74.787       33.64      141.92      90.578]
-1.5769607
[        130      155.38      243.08      284.61]
[     111.97      42.935      130.34      60.645]
-0.0021629776
[     111.85       28.52      160.37       57.04]
[      152.6       41.18      279.67      108.21]
0.012603099
[     34.145      125.17      95.175      199.65]
[     25.288       89.25      162.05      194.25]
0.28700206
[      162.3      271.55      244.62       347.7]
[      231.2       249.8         302       332.4]
0.072247595
None
[     33.445      24.485      102.76      93.455]
[      107.8      167.91      147.88       202.3]
0.064958215
[         82      224.67         202      351.33]
[      89.95        90.3      178.06   

In [7]:
import numpy as np
import supervision

def intersection_over_union(boxes1, boxes2):
  return supervision.utils.box

In [6]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

CONFIDENCE_TRESHOLD = 0.5
predictions = {}

result = list(best_model.predict("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/100_jpg.rf.08010ef293b05e01c68a3cba517d5260.jpg", conf=CONFIDENCE_TRESHOLD))[0]
detections = sv.Detections(
    xyxy=ultralytics.utils.ops.xyxy2xywh(result.prediction.bboxes_xyxy),
    confidence=result.prediction.confidence,
    class_id=result.prediction.labels.astype(int)
)
#print(detections.xyxy)

#iou(detections_list[0], prediction_list[0])
print(detections_list, prediction_list)


[2024-01-04 14:26:19] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


[array([     214.44,       32.22,         305,      112.22], dtype=float32), array([     263.85,      143.85,      403.08,      293.85], dtype=float32), array([      75.17,       65.52,      128.27,         120], dtype=float32), array([     106.55,       106.9,      172.07,      181.04], dtype=float32), array([     174.83,      108.96,      215.17,      149.65], dtype=float32), array([     184.62,      318.46,       242.3,      384.61], dtype=float32), array([     202.31,      263.85,      275.39,      393.08], dtype=float32), array([     143.79,      105.51,      182.41,      138.96], dtype=float32), array([     327.69,       272.3,      383.07,      323.08], dtype=float32), array([     71.335,      38.665,      150.66,      92.995], dtype=float32), array([     186.67,      85.005,      399.17,      283.34], dtype=float32), array([     195.38,      183.85,       267.7,      260.77], dtype=float32), array([     118.46,      129.99,      241.54,       242.3], dtype=float32), array([    